In [1]:
import matplotlib
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

from tqdm import trange, tqdm
from sklearn.metrics import accuracy_score
from sklearn import svm
import sklearn

import parmap
import torch


Autosaving every 180 seconds


In [2]:
#data_dir = '/media/cat/4TBSSD/yuki/'
data_dir = '/media/cat/4TBSSD/yuki/'


In [3]:
#############################################################



def expand(r1, r2):
    r1 = torch.from_numpy(r1.transpose(0,2,1))
    r2 = torch.from_numpy(r2)
    r3 = torch.matmul(r1, r2)
    
    del r1
    del r2
    torch.cuda.empty_cache()
    
    return r3

def load_trial(root_dir, session, subsample):
    r1 = np.load(root_dir+session+"/"+session+"_code_04_random_ROItimeCourses_30sec_pca_0.95.npy")[:,:,:900]
    r2 = np.load(root_dir+session+"/"+session+"_code_04_random_ROItimeCourses_30sec_pca_0.95_spatial.npy")
    print (r1.shape)

    if subsample:
        idx = np.random.choice(np.arange(r1.shape[0]), 10,replace=False)
        r1 = r1[idx]
        
    r3 = expand(r1,r2)   
    
    t1 = np.load(root_dir+session+"/"+session+"_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy")[:,:,:900]
    t2 = np.load(root_dir+session+"/"+session+"_code_04_trial_ROItimeCourses_30sec_pca_0.95_spatial.npy")
    if subsample:
        t1 = t1[idx]
    t3 = expand(t1,t2)
    
    return r3, t3

# 
sessions = ['IJ2pm_Feb29_30Hz','IJ2pm_Mar1_30Hz','IJ2pm_Mar2_30Hz','IJ2pm_Mar3_30Hz','IJ2am_Mar7_30Hz' ]

r = np.zeros((0,900,16384),'float32')
#t = np.zeros((0,900,16384),'float32')
subsample= True
for session in sessions:
    root_dir = '/media/cat/4TBSSD/yuki/IJ2/tif_files/'
    a, b = load_trial(root_dir, session, subsample)
    r = np.vstack((r,a))
    r = np.vstack((r,b))
    print ("done session: ", session)

print ("Total stack: ", r.shape)

(94, 7, 900)
done session:  IJ2pm_Feb29_30Hz
(61, 6, 900)
done session:  IJ2pm_Mar1_30Hz
(106, 6, 900)
done session:  IJ2pm_Mar2_30Hz
(48, 6, 900)
done session:  IJ2pm_Mar3_30Hz
(34, 6, 900)
done session:  IJ2am_Mar7_30Hz
Total stack:  (100, 900, 16384)


In [4]:
#############################################
###### PCA DECOMP OF CONCATENATED STACKS ####
#############################################

from sklearn.decomposition import PCA

print (r.shape)

X = r.reshape(r.shape[0]*r.shape[1],
                             r.shape[2])
print (X.shape)

# subselect data
n_selected = min(2500, X.shape[0])
idx = np.random.choice(np.arange(X.shape[0]),n_selected,
                       replace=False)
X_select = X[idx]

pca = PCA()
pca.fit(X_select)

import pickle as pk
fname_pca = '/home/cat/pca.pkl'
pk.dump(pca, open(fname_pca,"wb"))


# compute # of components needed for reconsturction to the requierd limit
expl_variance = pca.explained_variance_
print (expl_variance)
expl_variance = expl_variance/expl_variance.sum(0)
print (expl_variance)
sums = 0
pca_explained_var_val = 0.99
for k in range(expl_variance.shape[0]):
    sums+=expl_variance[k]
    if sums>=pca_explained_var_val:
        nComp = k+1
        break
print ("nComp required: ", k)

nComp = k
print ("May wish to add at least another few components")

(100, 900, 16384)
(90000, 16384)
[7.29104195e+00 1.85702437e+00 2.93111862e-01 ... 4.03295427e-32
 4.03295427e-32 4.13368426e-33]
[7.33914819e-01 1.86927701e-01 2.95045811e-02 ... 4.05956367e-33
 4.05956367e-33 4.16095827e-34]
nComp required:  5
May wish to add at least another few components


In [5]:
################################################
###### PCA DENOISING OF INDIVIDUAL SESSIONS ####
################################################

def pca_denoise_data(pca, data_stm):
    #print ("DATA_STM: ", data_stm.shape)
    X = data_stm.reshape(data_stm.shape[0]*data_stm.shape[1],
                         data_stm.shape[2])

    #
    time_filters = pca.transform(X)[:,:nComp]
    pca_time_filters = time_filters.reshape(data_stm.shape[0],
                                                 data_stm.shape[1],
                                                 -1).transpose(0,2,1)
    pca_spatial_filters = pca.components_[:nComp,:]



    return pca_time_filters, pca_spatial_filters

#sessions = ['IJ2pm_Feb29_30Hz','IJ2pm_Mar1_30Hz','IJ2pm_Mar2_30Hz','IJ2pm_Mar3_30Hz','IJ2am_Mar7_30Hz' ]
sessions = ['IJ2pm_Feb29_30Hz','IJ2pm_Mar1_30Hz'] #,'IJ2pm_Mar2_30Hz','IJ2pm_Mar3_30Hz','IJ2am_Mar7_30Hz' ]
save_dir = '/media/cat/4TBSSD/yuki/IJ2/concatenation_tests/'

# 
subsample=False
r = np.zeros((0,900,16384),'float32')
t = np.zeros((0,900,16384),'float32')
for session in sessions:
    root_dir = '/media/cat/4TBSSD/yuki/IJ2/tif_files/'
    a, b = load_trial(root_dir, session, subsample)
    
    t1, t2 = pca_denoise_data(pca, a)
    np.save(save_dir + session+"_trial.npy", t1)
    np.save(save_dir + session+"_trial_space.npy", t2)
    
    r1, r2 = pca_denoise_data(pca, b)
    np.save(save_dir + session+"_random.npy", r1)
    np.save(save_dir + session+"_random_space.npy", r2)



(94, 7, 900)
(61, 6, 900)
(106, 6, 900)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/media/cat/4TBSSD/anaconda3/envs/wide/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-8d94d4c8f982>", line 36, in <module>
    r1, r2 = pca_denoise_data(pca, b)
  File "<ipython-input-5-8d94d4c8f982>", line 11, in pca_denoise_data
    time_filters = pca.transform(X)[:,:nComp]
  File "/media/cat/4TBSSD/anaconda3/envs/wide/lib/python3.8/site-packages/sklearn/decomposition/_base.py", line 117, in transform
    X = X - self.mean_
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/media/cat/4TBSSD/anaconda3/envs/wide/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above ex

TypeError: object of type 'NoneType' has no len()

In [6]:
################################################
###### CONCATENATING INDIVIDUAL SESSIONS #######
################################################
save_dir = '/media/cat/4TBSSD/yuki/IJ2/concatenation_tests/'

sessions = ['IJ2pm_Feb29_30Hz','IJ2pm_Mar1_30Hz','IJ2pm_Mar2_30Hz','IJ2pm_Mar3_30Hz','IJ2am_Mar7_30Hz' ]

trials = []
random = []
for session in sessions:
    trials.append(np.load(save_dir+session+"_trial.npy"))
    random.append(np.load(save_dir+session+"_random.npy"))
    
trials = np.vstack(trials)
random = np.vstack(random)
print (trials.shape, random.shape)

# 

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 5 and the array at index 3 has size 20

In [7]:
##########################################################
##### RUN SVM ON SINGLE SESSIONS OR CONCATENATED SETS ####
##########################################################

def run_svm_single_randomized_kFold(
                                   run_id,
                                   idx_trials_split, 
                                   idx_random_split,
                                   window, 
                                   method, 
                                   trials, 
                                   random):

    # train data excludes the run_id
    idx_trials = np.delete(np.arange(trials.shape[0]),
                               idx_trials_split[run_id])
    idx_random = np.delete(np.arange(random.shape[0]),
                               idx_random_split[run_id])
    
    # test data is the left over labels
    idx_trials_not = np.delete(np.arange(trials.shape[0]),idx_trials)
    idx_random_not = np.delete(np.arange(random.shape[0]),idx_random)
    
    # stack train data
    train = np.vstack((trials[idx_trials],random[idx_random]))
    labels_train = np.hstack((np.ones(trials[idx_trials].shape[0]),
                              np.zeros(random[idx_random].shape[0])))

    # stack test data
    test = np.vstack((trials[idx_trials_not], random[idx_random_not]))
    labels_test = np.hstack((np.ones(trials[idx_trials_not].shape[0]),
                             np.zeros(random[idx_random_not].shape[0])))

    # 
    accuracy2=[]
    labels2 = []
    pred2 = []
    for k in range(0, trials.shape[2]-window, 1):
        X = train#[:,:,:window]
        X = X[:,:,k:k+window]
        #if mean_filter:
        #    X = np.mean(X,2)

        X = X.reshape(train.shape[0],-1)

        # 
        y = labels_train
        
        #
        X = sklearn.preprocessing.scale(X)

        # 
        clf = svm.SVC(kernel=method)
        clf.fit(X, y)


        # test 
        X_test = test[:,:,k:k+window]

        X_test = X_test.reshape(X_test.shape[0],-1)
        X_test = sklearn.preprocessing.scale(X_test)
        # 
        y_pred = clf.predict(X_test)

        # 
        acc = accuracy_score(labels_test, y_pred)
        accuracy2.append(acc)
        labels2.append(labels_test)
        pred2.append(y_pred)

    accuracy2 = np.array(accuracy2)
    labels2 = np.array(labels2)
    pred2 = np.array(pred2)

    #print ("inner loop: accraucy: ", accuracy2.shape, labels2.shape, pred2.shape)
    return accuracy2, labels2, pred2
    

# 
def compute_accuracy_svm(trials, 
                         random, 
                         data_split, 
                         window,
                         method,
                         xvalidation):

    
    # randomize seed
    np.random.seed()

    # select groups
    # NOTE: May wish to randomize here
    run_ids = np.arange(xvalidation)
        
    idx_trials_split = np.array_split(np.random.choice(np.arange(trials.shape[0]),
                                                       trials.shape[0],
                                                       replace=False),
                                     xvalidation)
    
    idx_random_split = np.array_split(np.random.choice(np.arange(random.shape[0]),
                                                       random.shape[0],
                                                       replace=False),
                                     xvalidation)
    
    data = parmap.map(run_svm_single_randomized_kFold,
                                       run_ids,
                                       idx_trials_split, 
                                       idx_random_split,
                                       window, 
                                       method, 
                                       trials, 
                                       random,
                                       pm_processes = 10,
                                       pm_pbar=True)
    
    # 
    accuracy = []
    labels = []
    predictions = []
    for k in range(len(data)):
        accuracy.append(data[k][0].T)
        labels.append(data[k][1].T)
        predictions.append(data[k][2].T)
      
    accuracy = np.vstack(accuracy).T
    labels = np.vstack(labels).T
    predictions = np.vstack(predictions).T
    
    
    return accuracy, labels, predictions

#

# 
use_concatenated = False
if use_concatenated:
    sessions = ['concatenated']
    trials_array = [trials]
    random_array = [random]
    pass

else:
    sessions = ['IJ2pm_Feb29_30Hz',
                'IJ2pm_Mar1_30Hz',
                'IJ2pm_Mar2_30Hz',
                'IJ2pm_Mar3_30Hz',
                'IJ2am_Mar7_30Hz' ]

    random_array = []
    trials_array = []
    for session in sessions:
        temp = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/'+session+
                      '/'+session+'_code_04_random_ROItimeCourses_30sec_pca_0.95.npy')
        
        temp = temp[:,:,:temp.shape[2]//2]
        random_array.append(temp)
        
        temp = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/'+session+
                      '/'+session+'_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
        temp = temp[:,:,:temp.shape[2]//2]
        trials_array.append(temp)



ctr_sess = 0
for session in sessions:
    
    trials = trials_array[ctr_sess]
    random = random_array[ctr_sess]
    print (trials.shape, random.shape)
    
    # 
    max_trials = min(trials.shape[0],random.shape[0])
    trials = trials[:max_trials]
    random = random[:max_trials]

    # 
    print ("trials: ", trials.shape, " random: ", random.shape)

    #
    clrs = ['black','blue','red','green','cyan','brown','pink','magenta','grey','yellow']

    #
    data_split = 0.8

    windows = [30]
    ctr = 0
    methods = ['sigmoid']
    xvalid = 5

    # 
    for ctr, window in enumerate(windows):
        accuracy, labels, predictions = compute_accuracy_svm(trials, 
                                                     random, 
                                                     data_split, 
                                                     window, 
                                                     methods[ctr],
                                                     xvalid)


        fname_out = os.path.join('/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/',
                             'SVM_Scores_'+
                             sessions[ctr_sess]+
                             'code_04'+
                             '_trial_ROItimeCourses_'+
                             str(window)+'sec'+
                             "_Xvalid"+str(xvalid)+
                             "_Slidewindow"+str(window)+
                             '.npz'
                             )
        np.savez(fname_out, 
                accuracy = accuracy,
                labels = labels,
                predictions = predictions)  

    
    ctr_sess+=1

(94, 7, 900) (94, 7, 900)
trials:  (94, 7, 900)  random:  (94, 7, 900)


100%|██████████| 5/5 [00:07<00:00,  1.42s/it]


(61, 6, 900) (61, 6, 900)
trials:  (61, 6, 900)  random:  (61, 6, 900)


100%|██████████| 5/5 [00:04<00:00,  1.14it/s]


(106, 6, 900) (106, 6, 900)
trials:  (106, 6, 900)  random:  (106, 6, 900)


100%|██████████| 5/5 [00:07<00:00,  1.55s/it]


(48, 6, 900) (48, 6, 900)
trials:  (48, 6, 900)  random:  (48, 6, 900)


100%|██████████| 5/5 [00:03<00:00,  1.35it/s]


(34, 6, 900) (34, 6, 900)
trials:  (34, 6, 900)  random:  (34, 6, 900)


100%|██████████| 5/5 [00:03<00:00,  1.61it/s]


In [8]:
data = np.load('/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Feb29_30Hzcode_04_trial_ROItimeCourses_30sec_Xvalid5_Slidewindow30.npz', allow_pickle=True)
data = np.load('/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar2_30Hzcode_04_trial_ROItimeCourses_30sec_Xvalid5_Slidewindow30.npz', allow_pickle=True)
data = np.load('/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar3_30Hzcode_04_trial_ROItimeCourses_30sec_Xvalid5_Slidewindow30.npz')
labels = data['labels']
print ("labels: ", labels.shape)

pred = data['predictions']
print ("pred: ", pred.shape)

clrs = ['black','blue']
trial_selected = 13

# # 
# ax=plt.subplot(121)
# t=np.arange(labels.shape[0])/30-30.

# for trial in range(labels.shape[1]):
#     plt.plot(t,pred[:,trial].cumsum(), 
#          c= clrs[int(labels[0,trial])])
#     if trial ==trial_selected:
#         plt.plot(t,pred[:,trial].cumsum(), 
#          c= 'red',
#                 linewidth=5,
#                 label="example trial label: "+str(labels[0,trial_selected]))
    
# plt.plot([t[0],t[-1]],[0,870],'--',c='blue',linewidth=10,
#          label="perfect decode leverpull", alpha=.5)
# plt.plot([t[0],t[-1]],[0,0],'--',c='black',linewidth=10,
#          label="perfect decode random", alpha=.5)
# plt.legend()

ax=plt.subplot(111)
t=np.arange(300)/30-10.

for trial in range(labels.shape[1]):
    plt.plot(t,pred[-300:,trial].cumsum(), 
         c= clrs[int(labels[0,trial])])
    if trial ==trial_selected:
        plt.plot(t,pred[-300:,trial].cumsum(), 
         c= 'red',
                linewidth=5,
                label="example trial label: "+str(labels[0,trial_selected]))
    
plt.plot([t[0],t[-1]],[0,300],'--',c='blue',linewidth=10,
         label="perfect decode leverpull", alpha=.5)
plt.plot([t[0],t[-1]],[0,0],'--',c='black',linewidth=10,
         label="perfect decode random", alpha=.5)
plt.legend()

plt.show()


labels:  (870, 96)
pred:  (870, 96)


In [9]:
###################################################
########## VISUALIZE SVM FOR EACH TRIAL ###########
###################################################
clrs = ['black','blue','red','green','cyan','brown','pink','magenta','grey','yellow']
fig=plt.figure()
ax=plt.subplot(121)
plt.plot(array_true.T.sum(1))

ax=plt.subplot(122)
plt.plot(array_false.T.sum(1))
plt.show()
plt.suptitle("accuracy for each class")    

# 
fig=plt.figure()

ax=plt.subplot(121)
for k in range(array_true.shape[0]):
    plt.plot(np.cumsum(array_true[k]), c=clrs[k])
plt.ylim(0,420)
ax=plt.subplot(122)
for k in range(array_false.shape[0]):
    plt.plot(np.cumsum(array_false[k]), c=clrs[k])
#plt.ylim(0,420)
plt.suptitle("cumulative prediction at each time point for both classes")    
plt.show()

#

array1 = array_true.copy()
array2 = array_false.copy()

idx = np.where(array1==0)
array1[idx]=-1
idx = np.where(array2==0)
array2[idx]=-1

NameError: name 'array_true' is not defined

In [10]:
root_dir = '/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/'

sessions = ['IJ2pm_Feb29_30Hz','IJ2pm_Mar1_30Hz','IJ2pm_Mar2_30Hz','IJ2pm_Mar3_30Hz','IJ2am_Mar7_30Hz' ]

ctr=1
n_trials = [90,58,99,43,33]
for session in sessions:
    ax=plt.subplot(2,3,ctr)
    
    data = np.load(root_dir + "SVM_Scores_"+session+"code_04_trial_ROItimeCourses_30sec_Xvalid5_Slidewindow30.npz")

    data = data['accuracy']
    # 
    mean = data.mean(1)
    std = np.std(data,1)
    t = np.arange(mean.shape[0])/30-29
    
    # 
    plt.plot(t, mean, c='blue')
    plt.fill_between(t, mean+std, mean-std, color='blue', alpha=.2,
                label=session)
                   
    data =np.load('/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_concatenatedcode_04_trial_ROItimeCourses_30sec_Xvalid5_Slidewindow30.npy')
    mean = data.mean(1)
    std = np.std(data,1)
    t = np.arange(mean.shape[0])/30-29
    
    # 
    plt.plot(t, mean, c='black')
    plt.fill_between(t, mean+std, mean-std, color='black', alpha=.2,
                label='concatenated')
    plt.legend()
    plt.title(session+ "  # trials: " +str(n_trials[ctr-1]))
    plt.ylim(0.4, 1.0)
    plt.xlim(-29,0)
    plt.plot([-29,0],[0.5,0.5],'r--')
    ctr+=1
print (data.shape)

FileNotFoundError: [Errno 2] No such file or directory: '/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_concatenatedcode_04_trial_ROItimeCourses_30sec_Xvalid5_Slidewindow30.npy'

In [12]:
################################################
###### MODEL RESULTS AS LEAKY INTEGRATORS ######
################################################
def model(array, ax, params, plotting=False):
    
    
    t=(np.arange(0,array.shape[1],1)-420)/30.
    
    # 
    prediction = np.zeros((array.shape),'float32')
    for k in range(array.shape[0]):
        #cs = np.zeros(array[k].shape[0], 'float32')
        for p in range(1,array[k].shape[0],1):

            # grab prediction
            temp = array[k][p]*params.scaling

            # urgency
            if params.urgency:
                #temp = temp*((k+urgency_scaling)/(urgency_scaling*2))
                temp = temp*(p**params.urgency_scaling/(
                                array[k].shape[0]**params.urgency_scaling))
                #print (k,p,temp)

            # cumulative part
            temp = temp+prediction[k,p-1]

            # noise comp
            if noise:
                temp = temp + np.random.normal(0, params.noise_sigma)

            # leaky component
            if leaky:
                temp = temp - np.sign(temp)*abs(np.random.normal(0, params.leaky_sigma))

            # check if threshold reached
            if abs(temp)>=params.threshold:
                prediction[k,p:]=temp
                break
            else:
                prediction[k,p]=temp 

        if plotting:
            ax.plot(t,prediction[k], c=clrs[k])
        
    return prediction

def compute_accuracy(pred1, pred2):
    
    print (pred1.shape)
    true_pos = np.zeros(pred1.shape[1],'float32')
    false_neg = np.zeros(pred1.shape[1],'float32')
    for k in range(pred1.shape[1]):
        idx = np.where(pred1[:,k]>0)[0]
        true_pos[k] = idx.shape[0]
        false_neg[k] = (pred1.shape[0]-true_pos[k])
            
    false_pos = np.zeros(pred2.shape[1],'float32')
    true_neg = np.zeros(pred2.shape[1],'float32')
    for k in range(pred2.shape[1]):
        idx = np.where(pred2[:,k]>0)[0]
        false_pos[k] = idx.shape[0]
        true_neg[k] = (pred2.shape[0]-false_pos[k])
    
    # compute sensitivity:  true positives / (true positives + false negatives)
    #sens.append(true_pos / (true_pos+false_neg))
    #spec.append(true_neg / (true_neg+false_pos))

    all_pos = true_pos + false_neg
    all_neg = false_pos + true_neg
    
    # compute accuracy
    accuracy = (true_pos+true_neg)/(all_pos+all_neg)
    
    return accuracy

#  class to hold the parameters
class params():
    
    def __init__(self):
        pass

# 
fig=plt.figure()
for k in range(10):
    p = params
    p.scaling = 0.1
    p.noise_sigma = 0.0
    p.leaky_sigma = 0.05
    p.threshold = 5.0
    p.urgency = True
    p.urgency_scaling = 1.+k*0.2

    #
    p.noise = False
    p.leaky = True
    p.plotting = False

    # 
    ax=[]
    if p.plotting:
        fig=plt.figure()
        ax=plt.subplot(121)
    pred1 = model(array1, ax, p)

    # 
    ax2=[]
    if p.plotting:
        ax2=plt.subplot(122)
    pred2 = model(array2, ax2, p)

    print ("..>DONE...")


    accuracy = compute_accuracy(pred1,pred2)
    t=(np.arange(0,pred1.shape[1],1)-420)/30.
    plt.plot(t, accuracy, c=clrs[k], linewidth=4, alpha=.4)

    
plt.plot(t ,accuracy_svm, c='black',linewidth=4)
plt.ylim(0.3,1.0)
plt.xlim(-15,0)
plt.plot([-15,0],[0.5,0.5])
plt.show()


NameError: name 'array1' is not defined

In [205]:

# 

    
#

(10, 420)


In [33]:
#####################################
##### PREDICT SVM DECISION TIME #####
#####################################
svm = PredictSVMTime()
svm.main_dir = '/media/cat/4TBSSD/yuki/'

# default params
svm.lockout = False
svm.lockout_window = 10
svm.pca_flag = True
svm.pca_var = 0.95
svm.window = 10
svm.use_saved_model = False
svm.fname_saved_model = '/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Feb11_30Hz/analysis/00599_svm.pkl'

# 
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2'] # "AR4" and other datasets could work

# 
lockouts = [False, True]
for lockout in lockouts:
    svm.lockout=lockout
    for id_ in names:
        svm.animal_id = id_
        svm.session_id = 'all'

        # 
        svm.predict()

# svm.process_data()

NameError: name 'PredictSVMTime' is not defined

In [18]:
################################################
################################################
################################################
fig = plt.figure()
svm.animal_id = 'AQ2'
svm.session_id = 'all'
svm.get_sessions()
print (svm.sessions)

#
ctr=1
for session_id in svm.sessions:
    ax=plt.subplot(8,10,ctr)
    svm.session_id = session_id
    try:
        svm.plot_decision_time(ax)
        plt.title(session_id+", #: "+str(svm.n_trials), fontsize=6)
        if ctr!=1:
            plt.yticks([])
        if ctr!=31:
            plt.xticks([])

        ctr+=1

    except:
        pass
plt.suptitle(svm.animal_id,fontsize=20)

['AQ2am_Dec9_30Hz' 'AQ2am_Dec10_30Hz' 'AQ2pm_Dec10_30Hz'
 'AQ2am_Dec11_30Hz' 'AQ2pm_Dec14_30Hz' 'AQ2am_Dec14_30Hz'
 'AQ2pm_Dec16_30Hz' 'AQ2am_Dec17_30Hz' 'AQ2pm_Dec17_30Hz'
 'AQ2am_Dec18_30Hz' 'AQ2pm_Dec18_30Hz' 'AQ2am_Dec21_30Hz'
 'AQ2am_Dec22_30Hz' 'AQ2am_Dec23_30Hz' 'AQ2am_Dec28_30Hz'
 'AQ2am_Dec29_30Hz' 'AQ2am_Dec30_30Hz' 'AQ2am_Dec31_30Hz'
 'AQ2am_Jan4_30Hz' 'AQ2am_Jan5_30Hz' 'AQ2am_Jan6_30Hz' 'AQ2am_Jan7_30Hz'
 'AQ2am_Jan8_30Hz' 'AQ2pm_Jan11_30Hz' 'AQ2am_Jan11_30Hz'
 'AQ2pm_Jan12_30Hz' 'AQ2am_Jan12_30Hz' 'AQ2pm_Jan13_30Hz'
 'AQ2am_Jan13_30Hz' 'AQ2pm_Jan14_30Hz' 'AQ2am_Jan14_30Hz'
 'AQ2am_Jan15_30Hz' 'AQ2pm_Jan15_30Hz' 'AQ2pm_Jan18_30Hz'
 'AQ2am_Jan18_30Hz' 'AQ2am_Jan19_30Hz' 'AQ2pm_Jan19_30Hz'
 'AQ2pm_Jan20_30Hz' 'AQ2am_Jan20_30Hz' 'AQ2pm_Jan21_30Hz'
 'AQ2am_Jan21_30Hz' 'AQ2pm_Jan22_30Hz' 'AQ2am_Jan22_30Hz'
 'AQ2am_Jan25_30Hz' 'AQ2pm_Jan25_30Hz' 'AQ2pm_Jan26_30Hz'
 'AQ2am_Jan26_30Hz' 'AQ2am_Jan27_30Hz' 'AQ2pm_Jan27_30Hz'
 'AQ2am_Jan28_30Hz' 'AQ2pm_Jan28_30Hz' 'AQ2am_Jan29_30Hz'
 

Text(0.5, 0.98, 'AQ2')

In [ ]:
names = ['IA1','IA2','IA3','IJ1','AQ2'] # "AR4" and other datasets could work

for name in names:
    rnn.animal_id = name
    rnn.session_id = 'all'

    rnn.run_rnn()

In [12]:
#
data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/Mar3_200_0.0001.npy.npz')
data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/IJ2pm_Mar2_30Hz_1000_0.0001.npz')
#data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/IJ2pm_Feb29_30Hz_1000_0.0001.npz')
b = data1['b_rnn']
c = data1['c_s']

#
print (b.shape)
mean = b.mean(1)
t=np.linspace(-9.5, 0, mean.shape[0])

ax=plt.subplot(111)

plt.plot(t, mean, c='black')
plt.fill_between(t, mean+c, mean-c, color='black', alpha=.2,
                label='RNN - 1000epochs')

#
data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/IJ2pm_Mar2_30Hz_200_0.0001.npz')
#data1 = np.load('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/IJ2pm_Feb29_30Hz_200_0.0001.npz')
b = data1['b_rnn']
c = data1['c_s']

#
print (b.shape)
mean = b.mean(1)
t=np.linspace(-9.5, 0, mean.shape[0])
plt.plot(t, mean, c='red')
plt.fill_between(t, mean+c, mean-c, color='red', alpha=.2,
                label='RNN - 200epochs')
#
if False:
    fname_svm = '/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar2_30Hz_code_04_trial_ROItimeCourses_10sec_pca_0.95.npy'
    data2 = np.load(fname_svm)
    print (data2.shape)
    data2 = data2[:300]
    mean = data2.mean(1)
    std = data2.std(1)
    t= np.linspace(t[0],t[-1],mean.shape[0])
    plt.plot(t, mean, 'blue')
    plt.fill_between(t, mean+std, mean-std, color='blue', alpha=.2,
                    label='SVM - 2 frames sliding window')

plt.legend(fontsize=20)
plt.xlim(-10,0)
plt.ylim(0.4, 1.0)
plt.plot([-10,0],[0.5,0.5],'r--',c='black')
ax.tick_params(axis='both', which='major', labelsize=20)
#plt.suptitle(os.path.split(fname_svm)[1])
plt.xlabel("Time (sec)",fontsize=20)
plt.ylabel("Decoding accuracy", fontsize=20)

plt.show()

(10, 10)
(10, 10)


<ipython-input-12-e7b69b6f7608>:48: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "r--" (-> color='r'). The keyword argument will take precedence.
  plt.plot([-10,0],[0.5,0.5],'r--',c='black')


In [ ]:
OLDER CODE BELOW

In [11]:
#############################################
############## VISUALIZE RESULTS ############
#############################################
    
# LEVER PULL
vis = Visualize()

# lever-related data
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 10
vis.lockout_window = 10
vis.code = 'code_04'
vis.lockout = False

vis.animal_id = "IA1"
vis.session_id = "Feb23_"
title = vis.animal_id + "  "+vis.session_id

#############################################
############## DECISION TYPE ################
#############################################
# select animal and session

# load pca 
fig =plt.figure()

# ax=plt.subplot(2,3,1)
# vis.pca_flag = False
# vis.lockout = True
# vis.pca_var = 0.99
# vis.plot_decision_choice('black',str(vis.pca_var),title, ax)

# vis.lockout = True
# vis.plot_decision_choice('blue',str(vis.pca_var)+" lockout",title, ax)

# 
# vis.lockout = True
# vis.plot_decision_choice('blue',str(vis.pca_var) + " lockout",title, ax)


# 
vars_ = [0.95]
for k in range(len(vars_)):
    #ax=plt.subplot(2,3,k+2)
    ax=plt.subplot(1,1,k+1)
    vis.pca_flag = True
    vis.lockout = False
    vis.pca_var = vars_[k]
    vis.plot_decision_choice('black',str(vis.pca_var),title, ax)

    # 
    vis.pca_flag = True
    vis.lockout = True
    vis.plot_decision_choice('blue',str(vis.pca_var)+" lockout",title, ax)

# vis.pca_flag = True
# vis.pca_var = 0.99
# ax=plt.subplot(2,3,2)
# vis.plot_decision_choice('blue','xxpcs_0.99Variance',title, ax)


# vis.pca_flag = True
# vis.pca_var = 0.95
# ax=plt.subplot(2,3,3)
# vis.plot_decision_choice('red','8pcs_0.95Variance',title, ax)

# vis.pca_flag = True
# vis.pca_var = 0.90
# ax=plt.subplot(2,3,4)
# vis.plot_decision_choice('green','4pcs_0.90Variance',title, ax)


# vis.pca_var = 0.50
# ax=plt.subplot(2,3,5)
# vis.plot_decision_choice('magenta','2pcs_0.50Variance',title, ax)


# fname = vis.main_dir+'/'+animal_id+'/SVM_scores_'+animal_id+'_lockout_'+str(session)+'.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('blue','lockout-denoised')


Self session: IA1pm_Feb23_30Hz
Self session: IA1pm_Feb23_30Hz


In [7]:
import pickle as pk

file = open('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_10sec_pca.pkl', 'rb')

# dump information to that file
pca = pk.load(file)

print (pca.components_.shape)


(1500, 16384)


In [9]:
data_stm = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_10.0sec_butterworth_globalAverage_0.1hz_6.0hz_04code_stm.npy')
print (data_stm.shape)

(62, 601, 128, 128)


In [10]:
# 
X = data_stm.reshape(data_stm.shape[0]*data_stm.shape[1],
                     data_stm.shape[2]*data_stm.shape[3])

mu= np.mean(X, axis=0)


NameError: name 'nComp' is not defined

In [11]:
nComp = 9
Xnew = np.dot(pca.transform(X)[:,:nComp],
             pca.components_[:nComp,:])

Xnew+=mu

data_stm_denoised_reshaped = Xnew.reshape(data_stm.shape[0], data_stm.shape[1],
                                          data_stm.shape[2], data_stm.shape[3])


In [17]:
time_filters = pca.transform(X)[:,:nComp]
pca_time_filters_only = time_filters.reshape(data_stm.shape[0], data_stm.shape[1],-1).transpose(0,2,1)
print (pca_time_filters_only.shape)


(62, 9, 601)


(62, 601, 9)


In [21]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb23_30Hz/IA1pm_Feb23_30Hz_code_04_trial_ROItimeCourses_10sec_pca_0.5.npy')
print (data.shape)

(82, 2, 601)


In [76]:
# -*- coding: utf-8 -*-
"""Cat Data.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1r-NABuBJOITVlW03VvhsQivPNzmNp4yp
"""

import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold 
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn import svm
import pandas as pd
from sklearn.metrics import confusion_matrix  
import random
from scipy import stats
from matplotlib.pyplot import MultipleLocator
import os


jobid = os.getenv('SLURM_ARRAY_TASK_ID')
name='IA1_lockout'

data_leverpress = np.load('/media/cat/4TBSSD/yuki/temp/IA1/data_04.npy',allow_pickle=True)[0]
data_random = np.load('/media/cat/4TBSSD/yuki/temp/IA1/data_04_random.npy',allow_pickle=True)[0]

print (data_leverpress.shape)


(62, 35, 601)


In [77]:


#classification of time (10-class)
X_l=data_leverpress #  [n_trials, n_areas, n_time_steps]
X_l=X_l.transpose((0,2,1))


X_R_l= X_l.reshape(-1,X_l.shape[1]*X_l.shape[2])
normal_X_l = preprocessing.normalize(X_R_l)
n_X_l=normal_X_l.reshape(X_l.shape[0],X_l.shape[1],X_l.shape[2])
X_l=n_X_l

X_l=X_l[:,:300,:]
X_k=X_l[:,0:30,:]
X_k=X_k.reshape(X_k.shape[0],X_k.shape[1]*X_k.shape[2])
for i in range(30,271,30):
    X_t = X_l[:,i:i+30,:]
    X_t=X_t.reshape(X_t.shape[0],X_t.shape[1]*X_t.shape[2])
    X_k=np.concatenate((X_k,X_t),axis=0)

y_ct=np.zeros(data_leverpress.shape[0])
for i in range(1,10):
    Z_ct=i*np.ones(data_leverpress.shape[0])
    y_ct=np.concatenate((y_ct,Z_ct))

# 
X_tSVM=X_k # 10s * 30frams
y_tSVM=y_ct # labels

#10-fold confusion matrix
clf = svm.SVC() # Non-linear classifier
ten_svm=[]
conf_matrix_ten=[]
kf = KFold(n_splits=10,random_state=None, shuffle=True) 
for train_index, test_index in kf.split(X_tSVM):
    X_train_k, X_test_k = X_tSVM[train_index], X_tSVM[test_index] 
    y_train_k, y_test_k = y_tSVM[train_index], y_tSVM[test_index]
    clf.fit(X_train_k, y_train_k)
    score=clf.score(X_test_k, y_test_k)
    y_predicted=clf.predict(X_test_k)
    cm=confusion_matrix(y_test_k,y_predicted)
    confusion_m=cm.T # make each row be the prediction
    conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
    conf_matrix_norm = np.nan_to_num(conf_matrix_norm)
    ten_svm.append(score)
    conf_matrix_ten.append(conf_matrix_norm)
np.save('conf_10_'+str(name)+'_'+str(jobid)+'.npy',conf_matrix_ten)
sc=np.mean(conf_matrix_ten,axis=0) # mean of confusion matrix
np.save('conf_avg_'+str(name)+'_'+str(jobid)+'.npy',sc)

<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np

In [78]:

#Next is the plotting.

confusion_10=sc.diagonal()
np.save('diagonal_'+str(name)+'_'+str(jobid)+'.npy',confusion_10)
confusion_s=conf_matrix_ten

confusion_d=[]
for i in range(0,10):
    confusion_d.append(confusion_s[i].diagonal())
x_std=np.std(confusion_d,axis=0)/(10**0.5)
x_s=x_std


fig = plt.figure(figsize=(21.5,20), dpi=64, facecolor='white')
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
axes = plt.subplot(111)
axes.tick_params(axis ='both', which ='both', length = 10,width=4,pad=20)

bwith = 3 
ax = plt.gca()
ax.spines['bottom'].set_linewidth(bwith)
ax.spines['left'].set_linewidth(bwith)
ax.spines['top'].set_linewidth(bwith)
ax.spines['right'].set_linewidth(bwith)
l=15

Y=np.linspace(0,1,12)
X=np.ones(Y.size)
x=np.arange(-9.5,0.5,1)
plt.text(0.1, 0.2, 'Lever Pull', fontdict={'size': 70}, rotation=90)
plt.text(-12.95, 1/10, 'Chance', fontdict={'size': 60}, rotation=0)
plt.axhline(y=1/10, color='black', linestyle='--',linewidth=6)

plt.errorbar(x,confusion_10,x_s,c='red',alpha=0.6,marker='s', mec='red', ms=3, mew=3,label='SVM',linewidth=l,elinewidth=12)

plt.plot( (0+0)*X, Y, color='black',ls='--',linewidth=6)

plt.title('Decoding decision time',fontdict={'size': 110},pad=60)
plt.xlabel('Time before Lever Pull (s)',fontdict={'size': 110},labelpad=60)
plt.ylabel('Decoding Accuracy',fontdict={'size': 110},labelpad=180)
plt.ylim(ymax = 1.09)
ax=plt.gca()
ax.xaxis.set_major_locator(MultipleLocator(2))
ax.xaxis.set_minor_locator(MultipleLocator(1))
ax.yaxis.set_major_locator(MultipleLocator(0.2))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))

plt.xlim(xmin = -10.5)
plt.xlim(xmax = 0.9)

for i in range(0,10):
    a=confusion_10[i]
    b=x_s[i]
    n=10
    s=b
    t=(a-(1/10))/s
    df=n-1
    p = (1 - stats.t.cdf(t,df=df))
    
    if (p > 0.01 and p <= 0.05):
        plt.plot(-9.5+i, confusion_10[i]+x_std[i]+0.06, '*',c='black', ms=20)
        
    if (p > 0.00001 and p <= 0.01):
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.06,'*',c='black', ms=20)
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.08,'*',c='black', ms=20)
    if  p <= 0.00001:
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.06,'*',c='black', ms=20)
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.08,'*',c='black', ms=20)
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.10,'*',c='black', ms=20)

plt.show()
#plt.savefig('./time_svm_'+str(jobid)+'.jpg')
#plt.close()

<ipython-input-78-40a74f02ec5b>:57: RuntimeWarning: divide by zero encountered in double_scalars
  t=(a-(1/10))/s


In [10]:
data= np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar3_30Hz/IJ2pm_Mar3_30Hz_code_04_lockout_10sec_random_ROItimeCourses_10sec_pca_0.95_spatial.npy')
print (data.shape)

(7, 16384)
